In [4]:
import os
from scipy import io
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [5]:
def process_data():    
    #folder_path = r"C:\Users\1bout\OneDrive\SamaniLab\MESc Files\MLData\Results"
    folder_path = r"/home/deeplearningtower/Documents/MATLAB/JonahCode/MLTestData/FullResults/ClinicalResults"
    test_file_path = r"C:\Users\1bout\OneDrive\SamaniLab\MESc Files\403170837836d865566676a2b5ddade1.mat"
    num_files = 421 #currently working with set of 421

    # x_train = []
    # y_train = []

    # x_test = []
    # y_test = []
    num_files = 1936
    image_shape = (2500,256,2)
    images = np.empty((num_files,*image_shape),dtype=np.float64)
    labels = [None] * num_files

    i = 0
    numbenign = 0
    nummalignant = 0
    for file in os.listdir(folder_path):
        if os.fsdecode(file).endswith('.mat'):

            # Load data as a test
            try:
                mat_data = io.loadmat(folder_path + '//' + file)
            except FileNotFoundError as e:
                print(e)
            else:
                frame_one = mat_data['result'][0,0]['Frame1']
                frame_two = mat_data['result'][0,0]['Frame2']
                label = mat_data['result'][0,0]['label'][0] #Grab label
                tumor_mask = mat_data['result'][0,0]['image_information'][0,0]['tumor_mask'] #tumor_mask
                tumor_mask = tumor_mask.astype(bool) #Convert to bool matrix
                youngs_modulus_matrix = mat_data['result'][0,0]['image_information'][0,0]['YM_image'] #YM matrix

                tumor_mean_YM = np.mean(youngs_modulus_matrix[tumor_mask])    
                background_mean_YM = np.mean(youngs_modulus_matrix[~tumor_mask])

                stiffness_ratio = tumor_mean_YM/background_mean_YM

                # match label:
                #     case 'benign':
                # numbenign = numbenign + 1
                # if numbenign < 288: #360 benign files
                # appending = np.zeros((1,2500,256,2))
                # appending[:,:,:,0] = frame_one
                # appending[:,:,:,1] = frame_two
                appending = np.stack((frame_one, frame_two), axis=-1)
                images[i] = appending
                # images.append(appending)
                # x_train[numbenign-1,:,:,1] = frame_two

                labels[i] = int(stiffness_ratio)

                        # # else: #360 benign files
                        # appending = np.zeros((1,2500,256,2))
                        # appending[:,:,:,0] = frame_one
                        # appending[:,:,:,1] = frame_two
                        # x_test.append(appending)
                        # # x_train[numbenign-1,:,:,1] = frame_two

                        # y_test.append(int(stiffness_ratio))

                    # case 'malignant':
                # nummalignant = nummalignant + 1
                #         # if nummalignant < 59: #61 benign files
                # appending = np.zeros((1,2500,256,2))
                # appending[:,:,:,0] = frame_one
                # appending[:,:,:,1] = frame_two
                # images.append(appending)
                # # x_train[numbenign-1,:,:,1] = frame_two

                # labels.append(int(stiffness_ratio))

                        # else: #61 benign files
                        #     appending = np.zeros((1,2500,256,2))
                        #     appending[:,:,:,0] = frame_one
                        #     appending[:,:,:,1] = frame_two
                        #     x_test.append(appending)
                        #     # x_train[numbenign-1,:,:,1] = frame_two

                        #     y_test.append(int(stiffness_ratio))

                i = i + 1
                print('Done File ',i)
    
    print("Done Processing")
    print('num labels: ',len(set(labels)))
    # # Make list with both things
    # images = x_test
    # print('test 1')
    # images.extend(x_train)
    # print('test 2')
    # a = np.shape(images)
    # print('gopt shape')
    # print('Images shape: ',a)
    # labels = y_test
    # print('test 3')
    # labels.extend(y_train)
    # print('Labels shape: ',np.shape(labels))
    # print('test 4')

    # images_array = np.zeros((len(images),2500,256,2))
    # print('test 5')
    # for i in range(len(images)):
    #     print('Appending ',i)
    #     images_array[i,:,:,:] = images[i]
    # print('test 5')
    # images = images_array
    # print('test 7')
    # labels_array = np.zeros(len(labels))
    # print('test 8')
    # for i in range(len(labels)):
    #     print('Appending ',i)
    #     labels_array[i] = labels[i]
    # print('test 9')
    # labels = labels_array
    # print('test 10')
    return images,labels
images,labels = process_data()

Done File  1
Done File  2
Done File  3
Done File  4
Done File  5
Done File  6
Done File  7
Done File  8
Done File  9
Done File  10
Done File  11
Done File  12
Done File  13
Done File  14
Done File  15
Done File  16
Done File  17
Done File  18
Done File  19
Done File  20
Done File  21
Done File  22
Done File  23
Done File  24
Done File  25
Done File  26
Done File  27
Done File  28
Done File  29
Done File  30
Done File  31
Done File  32
Done File  33
Done File  34
Done File  35
Done File  36
Done File  37
Done File  38
Done File  39
Done File  40
Done File  41
Done File  42
Done File  43
Done File  44
Done File  45
Done File  46
Done File  47
Done File  48
Done File  49
Done File  50
Done File  51
Done File  52
Done File  53
Done File  54
Done File  55
Done File  56
Done File  57
Done File  58
Done File  59
Done File  60
Done File  61
Done File  62
Done File  63
Done File  64
Done File  65
Done File  66
Done File  67
Done File  68
Done File  69
Done File  70
Done File  71
Done File  72
D

: 

In [3]:
def load_data(images,labels,batch_size = 32):
    """
    Prepare the data for the model, once it's been processed
    """
    images_tensor = torch.tensor(images,dtype=torch.float64)
    labels_tensor = torch.tensor(labels,dtype=torch.long)

    dataset = TensorDataset(images_tensor,labels_tensor) #Dataset for model

    data_loader = DataLoader(dataset,batch_size=batch_size,shuffle=True) #Dataloader for batching
    return data_loader

In [4]:
class CNN(nn.Module):
    def __init__(self,num_classes = 2):
        super(CNN,self).__init__()

        #Conv layers
        self.conv1 = nn.Conv2d(in_channels=2,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.conv2 = nn.Conv2d(32,64,3,1,1)
        self.conv3 = nn.Conv2d(64,128,3,1,1)

        #Max pooling layer
        self.pool = nn.MaxPool2d(2,2)

        #Fully connected layer
        self.fc1 = nn.Linear(128*1250*128,512)
        self.fc2 = nn.Linear(512,num_classes)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x))) #Conv + ReLU + Pooling
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        return x


In [5]:
def initialize_model(num_classes = 2,learning_rate = 0.001):
    model = CNN(num_classes)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)

    return model,criterion,optimizer,device

In [ ]:
def train_model(model,data_loader,criterion,optimizer,device,num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_preds  = 0
        total_preds = 0

        for inputs, targets in data_loader:
            inputs,targets = inputs.to(device),targets.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)

            loss = criterion(outputs,targets)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _,predicted = torch.max(outputs,1)
            correct_preds += (predicted==targets).sum().item()
            total_preds += targets.size(0)

            epoch_loss = running_loss / len(data_loader)
            epoch_accuracy = correct_preds / total_preds * 100

            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

In [ ]:
def main():
    # images,labels = process_data()
    data_loader = load_data(images,labels,batch_size=32) #Preprocess data
    print('test')
    model,criterion,optimizer,device = initialize_model(num_classes=2)

    num_epochs = 10
    train_model(model,data_loader,criterion,optimizer,device,num_epochs)
main()

    